# Structural Balance
### Social Network Analysis - Group 9 Project
* Miriam Schwebler - 01000421
* Isaak Mengesha - 11945608
* Sabrina Kuhrn - 00926421
* Stefan Hödl - 01452750

In [ ]:
import numpy as np
from networkx import nx

import pandas as pd
import datetime
import matplotlib.pyplot as plt
from numpy.linalg import matrix_power

## The Data

The data set is provided by Der Standard, one of the top Austrian newspapers.
In the online Standard people can post comments below articles and up/down vote comments.
The data set used in this handson and further in the project part of the course will consider a sample of those articles, comments, and votes. 

In [ ]:
date_cols = ["PostingCreatedAt","ArticlePublishingDate"]

df1 = pd.read_csv('../data/Postings_01052019_15052019.csv',usecols=["ID_CommunityIdentity", "ID_Posting", "PostingCreatedAt", "ArticleTitle",'ArticleChannel' ,"ArticleRessortName","ArticlePublishingDate"],parse_dates=date_cols, sep=';')
df2 = pd.read_csv('../data/Postings_16052019_31052019.csv', usecols=["ID_CommunityIdentity", "ID_Posting","PostingCreatedAt", "ArticleTitle",'ArticleChannel' ,"ArticleRessortName","ArticlePublishingDate"], parse_dates=date_cols,sep=';')
df  = df1.append(df2, ignore_index=True)
df  = df[(df.ArticleChannel == "Inland") & (~df.ArticleRessortName.isin([ "Pensionen", "Eurofighter","Off-Topic"]))]
df.shape

There are different entities in the data set: 
* Users - identified by *ID_CommunityIdentity* (or *UserCommunityName*)
* Postings - identified by *ID_Posting*
* Articles - identified by *ID_Article*

Thus, there are different possibilities to build networks based on voting and posting data. 
We will concentrate now on the ***votes-to-network***. 


In [ ]:
date_cols = ["VoteCreatedAt","UserCreatedAt"]
votes1 = pd.read_csv('../data/Votes_01052019_15052019.csv',parse_dates=date_cols, sep=';')
votes2 = pd.read_csv('../data/Votes_16052019_31052019.csv', parse_dates=date_cols,sep=';')
votes=votes1.append(votes2, ignore_index=True)

PostAndVotes=pd.merge(df,votes,on="ID_Posting")
PostAndVotes.shape

In [ ]:
# filter off (= 1)
PostAndVotes_less=PostAndVotes.groupby('ID_Posting').filter(lambda x : len(x)>1).copy()
split_date= datetime.datetime(2019,5,17)

PostAndVotes_before = PostAndVotes_less.loc[PostAndVotes_less['PostingCreatedAt'] <= split_date]
PostAndVotes_after = PostAndVotes_less.loc[PostAndVotes_less['PostingCreatedAt'] > split_date]

#PostAndVotes_after.head()
print('Before shape: ' + str(PostAndVotes_before.shape))
print('After shape: ' + str(PostAndVotes_after.shape))

A line in the table above shows that a user (i.e., *ID_CommunityIdentiy*) posted a comment. Every post has its own uniqe identifier (i.e., *ID_Posting*). If a user votes for a posting then the vote is identified by the *ID_Posting* the voting was for, the *ID_CommunityIdentiy* from the voter. Next, it is also recorded, if the vote was negative or positive. This informtion is saved in  *VoteNegative* and *VotePositive* respectively.  

We want to bring the structure above into following format: 
* source, i.e., the voting user
* target, i.e., the post creator
* weight, i.e., how often the source voted for the target (postive and negative)

In other words, we are aiming for a *weighted edge-list*.

### Edges

In [ ]:
edgeListBefore= PostAndVotes_before.groupby(["ID_CommunityIdentity_x","ID_CommunityIdentity_y"]).agg({"VoteNegative": [("votes_neg_count","sum")], "VotePositive":[("votes_pos_count","sum")]})
edgeListAfter= PostAndVotes_after.groupby(["ID_CommunityIdentity_x","ID_CommunityIdentity_y"]).agg({"VoteNegative": [("votes_neg_count","sum")], "VotePositive":[("votes_pos_count","sum")]})

edgeListBefore.columns=edgeListBefore.columns.droplevel()
edgeListAfter.columns=edgeListAfter.columns.droplevel()

### Weight calculation

In [ ]:
## (-1 if any_neg_vote else 1)
edgeListBefore["weight"]= np.where(edgeListBefore["votes_neg_count"] > 0, -1, 1) 
edgeListAfter["weight"]= np.where(edgeListAfter["votes_neg_count"] > 0, -1, 1) 

## alternative
# edgeListBefore["weight"]= (1+edgeListBefore["votes_pos_count"])/(1+edgeListBefore["votes_neg_count"])
# edgeListAfter["weight"]=(1+edgeListAfter["votes_pos_count"])/(1+edgeListAfter["votes_neg_count"])

edgeListBefore.rename_axis(['source', 'target'], inplace=True)
edgeListAfter.rename_axis(['source', 'target'], inplace=True)
#edgeListAfter.describe()

In [ ]:
## write to file
edgesBefore = edgeListBefore.reset_index()
edgesAfter = edgeListAfter.reset_index()

edgesBefore.to_csv("../data/edges_before.csv", index=False)
edgesAfter.to_csv("../data/edges_after.csv", index=False)

### Construct Graph

We use the *networkx* library.
Since we build a *votes-to-network* we have *source* nodes and *target* nodes. 
Thus, the network is directed.
Therefore, we use *nx.Digraph()*

In [ ]:
G = nx.from_pandas_edgelist(edgesAfter, 
                            source='source', 
                            target='target', 
                            edge_attr = 'weight',
                            create_using=nx.DiGraph())

# Extract dense subgraph
* Create undirected Adjacency Matrix from diGraph 

* take its third power ($A^3$)

* diagonal captures in how many triads a node _might_ be part of

* store values in dict, which measure number of possible cycles of length 3 (= triad)

* extract subgraph with only nodes which might be part of a triad

* thus reduce number of nodes from 20K to 287

In [ ]:
## use undirected graph and convert to adjacency matrix
A = np.abs(nx.to_numpy_matrix(G.to_undirected()))

## take matrix to 3rd power
A3 = matrix_power(A,3)

In [ ]:
## store in a dictionary (A3di): diagonal & undirected & its a dict!
A3di = {}
for i in range(len(A3)):
    ## store node if potentially part in > 0 triads
    if (A3[i,i] > 0):
        A3di[i] = A3[i,i] 
print(len(A3di.keys()))

##  Make Sub-dictionary with only highly connected ones
A3di_mini = {k:v for k,v in A3di.items() if v > 200}
print(len(A3di_mini.keys()))

# write after dict to file
A3di_after_df = pd.DataFrame.from_dict(A3di, orient='index')
A3di_after_df.to_csv("../dicts/A3di_after.csv")

### Repeat for first half of dataset (before Ibiza)

In [ ]:
## construct directed graph, convert to undirected adjacency matrix
G_b = nx.from_pandas_edgelist(edgesBefore, 
                            source='source', 
                            target='target', 
                            edge_attr = 'weight',
                            create_using=nx.DiGraph())
Adj_b = nx.to_numpy_matrix(G_b.to_undirected())

## 3rd power
A3_b = matrix_power(Adj_b,3)

In [ ]:
A3di_b = {}
for i in range(len(A3_b)):
    if (A3_b[i,i] > 0):
        A3di_b[i] = A3_b[i,i] 
len(A3di_b.keys())

# write before dict to file
A3di_before_df = pd.DataFrame.from_dict(A3di_b, orient='index')
A3di_before_df.to_csv("../dicts/A3di_before.csv")

### Construct the subgraph which should be highly connected.

In [ ]:
# read before dictionary from file
A3di_before_df = pd.read_csv("../dicts/A3di_before.csv", index_col=0)
A3di_b = A3di_before_df['0'].to_dict()
# read after dictionary from file
A3di_after_df = pd.read_csv("../dicts/A3di_after.csv", index_col=0)
A3di = A3di_after_df['0'].to_dict()
print(len(A3di.keys()), len(A3di_b.keys()))

In [ ]:
## possibly filter for value > k
A3di_mini = {k:v for k,v in A3di.items() if v > 200}
len(A3di_mini)

## construct subgraph using dictionary
Gmini = G.subgraph(A3di_mini.keys()).copy()

# Triadic Census Algorithm

In [ ]:
from collections import defaultdict
from itertools import combinations
import nxtriads as nx2 ## import modified functions

In [ ]:
def find_double_edge(triad_edge_data):
    for i in triad_edge_data:
        for j in triad_edge_data:
            if (i[0]==j[1] and i[1]==j[0]):
                x = i[0]
                y = i[1]
    return x,y

def find_double_edge2(triad_edge_data):
    double_edges = []
    for i in triad_edge_data:
        for j in triad_edge_data:
            if (i[0]==j[1] and i[1]==j[0]):
                x = i[0]
                y = i[1]
                double_edges.append((x,y))
    return double_edges

def Reverse(tuples): 
    new_tup = tuples[::-1] 
    return new_tup 

In [ ]:
def triads_by_specific_types(G, my_types):
    """Returns a list of triads of pre-defined types in a directed graph.
    Parameters
    ----------
    G : digraph
       A NetworkX DiGraph
    Returns
    -------
    tri_by_type : dict
       Dictionary with triad types as keys and lists of triads as values.
    """
    all_tri = nx2.all_triads_mod(G) # modified function to skip if len < 3
    tri_by_type = defaultdict(list)
    for triad in all_tri:
        if (nx2.triad_type(triad) in my_types):
            name = nx2.triad_type(triad)
            tri_by_type[name].append(triad)
    return tri_by_type

def balance_for_2_semicycles(triad_edge_data):
    semicycle_balance = 1
    x, y = find_double_edge(triad_edge_data)
    positive_set = []
    negative_set = []
    
    # calculating balance/imbalance for both semicycles
    for edge in triad_edge_data:
        semicycle_balance *= (edge[2])['weight']
        
    for edge in triad_edge_data:
        if ((x == edge[0]) and (y == edge[1])):
            semicycle_balance_1 = semicycle_balance / ((edge[2])['weight'])
        if ((x == edge[1]) and (y == edge[0])):
            semicycle_balance_2 = semicycle_balance / ((edge[2])['weight'])
            
    # checking for both semicycles if balanced or imbalanced   
    if (semicycle_balance_1 > 0):
        positive_set.append(semicycle_balance_1)
    else:
        negative_set.append(semicycle_balance_1)
            
    if (semicycle_balance_2 > 0):
        positive_set.append(semicycle_balance_2)
    else:
        negative_set.append(semicycle_balance_2)
            
    triangle_balance = len(positive_set)/(len(positive_set) + len(negative_set))   # step 8 in algorithm             
    return triangle_balance

def semicycle_extraction(double_edges):
    triplets = combinations(double_edges, 3)
    triple = []
    cycles = []
    delete = []
    semicycles = []
    
    for triplet in triplets:
        triple.append(triplet)
        
    for i in range(len(triple)):
        if (Reverse((triple[i])[0]) in triple[i]):
            delete.append(triple[i])
        elif (Reverse((triple[i])[1]) in triple[i]):
            delete.append(triple[i])
        else:
            cycles.append(triple[i])
            
    nodes = []
    for tuples in cycles[0]:
        nodes.append(tuples[0])
        nodes.append(tuples[1])
    unique_nodes = np.unique(nodes)

    a = unique_nodes[0]
    b = unique_nodes[1]
    c = unique_nodes[2]
            
    for cycle in cycles:
        if ((a,b) in cycle and (b,c) in cycle and (c,a) in cycle):
            delete.append(cycle)
        elif ((b,a) in cycle and (c,b) in cycle and (a,c) in cycle):
            delete.append(cycle)
        else:
            semicycles.append(cycle)

    return semicycles

def balance_for_triad_type_300(triad_edge_data):
    semicycle_balance = 1
    double_edges = find_double_edge2(triad_edge_data)
    semicycles = semicycle_extraction(double_edges)
    
    positive_set = []
    negative_set = []    

    for cycle in semicycles:
        semicycle_balance = 1
        for i in range(3):
            for edge in triad_edge_data:
                if (cycle[i] == (edge[0], edge[1])):
                    semicycle_balance *= edge[2]['weight']
        if semicycle_balance > 0:
            positive_set.append(semicycle_balance)
        else:
            negative_set.append(semicycle_balance)
            
    triangle_balance = len(positive_set)/(len(positive_set) + len(negative_set))   # step 8 in algorithm             
    return triangle_balance

In [ ]:
def balance_ratio(graph):
    my_types = {'030T', '120D', '120U', '300'}
    triads = triads_by_specific_types(graph, my_types)
    
    sum_balance_030T = 0
    sum_balance_120D = 0
    sum_balance_120U = 0
    sum_balance_300  = 0
    
    rel_triads = {k:v for (k,v) in triads.items() if k in my_types}
    
    for triad_type in my_types:
        for i in range(len(triads[triad_type])):
            
            if (triad_type == '030T'):
                edge_data = ((triads['030T'])[i]).edges(data = True)
                semicycle_balance_030T = 1
                for edge in edge_data:
                    semicycle_balance_030T *= (edge[2])['weight']
                if (semicycle_balance_030T > 0):
                    sum_balance_030T += 1
                else:
                    sum_balance_030T += 0
                
            elif (triad_type == '120D'):
                edge_data = ((triads['120D'])[i]).edges(data = True)
                triangle_balance_120D = balance_for_2_semicycles(edge_data)
                
                sum_balance_120D += triangle_balance_120D
                
            elif (triad_type == '120U'):
                edge_data = ((triads['120U'])[i]).edges(data = True)
                triangle_balance_120U = balance_for_2_semicycles(edge_data)
                
                sum_balance_120U += triangle_balance_120U
                
            elif (triad_type == '300'):
                edge_data = ((triads['300'])[i]).edges(data = True)
                triangle_balance_300 = balance_for_triad_type_300(edge_data)
                
                sum_balance_300 += triangle_balance_300
            
    if len(triads['030T']) > 0:
        overall_balance_030T = sum_balance_030T / len(triads['030T'])
    else: 
        overall_balance_030T = 0
        
    if len(triads['120D']) > 0:
        overall_balance_120D = sum_balance_120D / len(triads['120D'])
    else: 
        overall_balance_120D = 0
        
    if len(triads['120U']) > 0:
        overall_balance_120U = sum_balance_120U / len(triads['120U'])
    else: 
        overall_balance_120U = 0
        
    if len(triads['300']) > 0:
        overall_balance_300 = sum_balance_300 / len(triads['300'])
    else: 
        overall_balance_300 = 0
    
    average_overall_balance = (overall_balance_030T + overall_balance_120D + overall_balance_120U + overall_balance_300) / 4

    return average_overall_balance, rel_triads

### run triadic census algorithm on extracted, dense Subgraph

In [ ]:
## run algorithm on graph after ibiza
average_overall_balance, rel_triads = balance_ratio(Gmini)
print(average_overall_balance)

In [ ]:
## run algorithm on graph before ibiza
average_overall_balance_before, rel_triads_before = balance_ratio(Gmini)
print(average_overall_balance_before)

### Plotting relevant subgraph

In [ ]:
## modified to just extract relevant triads
def get_rel_triads(graph):
    my_types = {'030T', '120D', '120U', '300'}
    triads = triads_by_specific_types(graph, my_types)

    return {k:v for (k,v) in triads.items() if k in my_types}

rel_triads = get_rel_triads(Gmini)

rel = []
rels = set()
for k,v in rel_triads.items():
    for i, tri in enumerate(v):
        rel.append(list(tri.nodes))
        for idx in list(tri.nodes):
            rels.add(idx)

In [ ]:
Gmicro = Gmini.subgraph(rels).copy()
Gmicro_adj = nx.to_numpy_matrix(Gmicro)

edges, weights = zip(*nx.get_edge_attributes(Gmicro,'weight').items())
nx.draw(Gmicro_adj, node_color='darkblue', edgelist=edges, edge_color=weights, width=2.0, edge_cmap=plt.cm.Spectral)

### ...

### Optional: extract Follow-Ignore Relationships 
* Extract ignore edges for all node-pairs in reduced graph - 

* only 7 ignore edges remain, so we do not include this in our data 

* since no timestamp is available, we cannot discern before/after ibiza

In [ ]:
follow = pd.read_csv(('../data/Following_Ignoring_Relationships_01052019_31052019.csv'), sep=';')
t1 = follow[follow.ID_CommunityConnectionType == 1]#.iloc[:,:2] # follow
t2 = follow[follow.ID_CommunityConnectionType == 2]#.iloc[:,:2] # ignore
t2.loc[:,'ID_CommunityConnectionType'] = t2.loc[:,'ID_CommunityConnectionType'].replace(2, -1)

In [ ]:
ignore = []
for _, (s,t) in enumerate(zip(t2.ID_CommunityIdentity, t2.ID_CommunityIdentityConnectedTo)):
    if (s in A3di.keys()) & (t in A3di.keys()):
        ignore.append((s,t))
len(ignore)

# Triangle Index & Walk Index

In [ ]:
import igraph as ig
import leidenalg as la
import cairo
import random
from itertools import combinations, groupby
from scipy.linalg import expm


def gnp_random_connected_graph(n, p):
    """
    Generates a random undirected graph, similarly to an Erdős-Rényi 
    graph, but enforcing that the resulting graph is conneted
    """
    edges = combinations(range(n), 2)
    G = nx.Graph()
    G.add_nodes_from(range(n))
    if p <= 0:
        return G
    if p >= 1:
        return nx.complete_graph(n, create_using=G)
    for _, node_edges in groupby(edges, key=lambda x: x[0]):
        node_edges = list(node_edges)
        random_edge = random.choice(node_edges)
        G.add_edge(*random_edge)
        for e in node_edges:
            if random.random() < p:
                G.add_edge(*e, weight=random.choice([1,-1]))
    return G


nodes = 200
probability = 0.1
G = gnp_random_connected_graph(nodes,probability)

plt.figure(figsize=(8,5))
nx.draw(G, node_color='lightblue', 
        with_labels=True, 
        node_size=500)

In [ ]:
A=nx.to_numpy_matrix(G)
A3=matrix_power(A,3)
A_absolut_3=matrix_power(abs(A),3)
#print(A_absolut_3)

triangle_index=(np.trace(A3)+np.trace(A_absolut_3))/(2*np.trace(A_absolut_3))
print(triangle_index)



In [ ]:
A=nx.to_numpy_matrix(G)
A_exp_trace=np.trace(expm(A))
A_abs_Exp_trace=np.trace(expm(abs(A)))
WBM=A_exp_trace/A_abs_Exp_trace #Walk-based measure


In [ ]:
A=nx.to_numpy_matrix(G)
H= ig.Graph.Adjacency((abs(A) > 0).tolist())
partition =la.find_partition(H, la.ModularityVertexPartition)
ig.plot(partition)


## BELOW: all old = remove?

In [ ]:
# edgesBefore = pd.read_csv("../data/votes_to_comments_before.csv")
# edgesAfter = pd.read_csv("../data/votes_to_comments_after.csv")

In [ ]:
# G = nx.from_pandas_edgelist(edgesAfter, 
#                             source='source', 
#                             target='target', 
#                             edge_attr = 'weight',
#                             create_using=nx.DiGraph())

In [ ]:
# UG = G.to_undirected()
# #count=0
# for node in G:
#     for ngbr in nx.neighbors(G, node):
#         if node in nx.neighbors(G, ngbr):
#             UG.edges[node, ngbr]['weight'] = (np.where( 
#                 G.edges[node, ngbr]['weight'] + G.edges[ngbr, node]['weight'] >=0,1,-1))
#            # if np.sign(G.edges[node, ngbr]['weight'])==np.sign(G.edges[ngbr, node]['weight']):
#                                        #       count=count+1

In [ ]:
# A = nx.to_numpy_matrix(UG)
# A3 = matrix_power(A,3)

In [ ]:
# A_absolut_3 = matrix_power(abs(A),3)
# triangle_index = (np.trace(A3) + np.trace(A_absolut_3)) / (2*np.trace(A_absolut_3))
# triangle_index

## Before

In [ ]:
# #edgesBefore = pd.read_csv("../data/votes_to_comments_before.csv")
# G_2 = nx.from_pandas_edgelist(edgesBefore, 
#                             source='source', 
#                             target='target', 
#                             edge_attr = 'weight',
#                             create_using=nx.DiGraph())

In [ ]:
# UG_2 = G_2.to_undirected()
# for node in G_2:
#     for ngbr in nx.neighbors(G_2, node):
#         if node in nx.neighbors(G_2, ngbr):
#             UG_2.edges[node, ngbr]['weight'] = (np.where( 
#                 G_2.edges[node, ngbr]['weight'] + G_2.edges[ngbr, node]['weight'] >=0,1,-1))
#            # if np.sign(G.edges[node, ngbr]['weight'])==np.sign(G.edges[ngbr, node]['weight']):
#                                        #       count=count+1

In [ ]:
# A = nx.to_numpy_matrix(UG_2)
# A3 = matrix_power(A,3)


In [ ]:
# A_absolut_3 = matrix_power(abs(A),3)
# triangle_index_2 = (np.trace(A3) + np.trace(A_absolut_3)) / (2*np.trace(A_absolut_3))
# triangle_index_2

__________________________________________